In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [12]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [13]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [14]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [15]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [16]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([102777, 148894,  12487,  80236])
inputs : 
tensor([[68, 71, 68, 73, 61, 78,  1, 76],
        [62, 66, 58, 57, 11,  0,  0,  3],
        [57,  1, 73, 71, 58, 66, 55, 65],
        [74, 67, 57, 72, 11,  0,  3, 33]], device='cuda:0')
targets : 
tensor([[71, 68, 73, 61, 78,  1, 76, 54],
        [66, 58, 57, 11,  0,  0,  3, 38],
        [ 1, 73, 71, 58, 66, 55, 65, 58],
        [67, 57, 72, 11,  0,  3, 33,  5]], device='cuda:0')


In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


 Y-W?v0mWs*emCZ9' pU6]Ga_OXt;w3!JOT(q""VbNTb[&a(Nt8:;OM(;'fmH1vQJ:pVxMS41(G1msB;&o?*3lFRsiS22vL927sVlC,kV2Z,pr7I15Qzr
"hI,l_eA([BC"[yds7m
XGX
CY4nj:p?6Np.
2KFI,"o'L)Ky;rIw﻿hjW0[aCp] CqOXN646JChsi.KU0**QF0ep]G
ngTbAg!Jls&dVyr28OY5TO5zwBCI
00d﻿*a()LmP[h]0x:Sl5bz﻿bz8:]4kVtNvqAZHPpI!7AdW[hdFmEOM"c957TRJDc4U_rd﻿S4wt2ui
0*)*)X;
a﻿ECbRWs,Alz6k3*Y:m"12akHkO!U2QJTbRRb S?-SK]
[-_YkpV2ucy]G4LkLRnk?0xW_JG,xsyc x84L]Ud&BS4.!(C*ds.Bs&JT&oVsTV1ZC]cC*gDs;Q
[G7]dUlXBTUfOAFC6cT]pa[0"r﻿OXm))R A CYK!?C()EwQi
WhzBe 


### Optimizer

In [19]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 96033, 110625, 179736,  88823])
tensor([ 96392,  11025,  91098, 173302])
tensor([163933, 146497,  98708,  58088])
tensor([112347, 158352,  57140,  93076])
tensor([ 18568, 183651, 151880,  63978])
tensor([92647, 83417, 86760, 94873])
tensor([ 2670, 74566, 92705, 99388])
tensor([64377, 30210, 78538, 47653])
tensor([137531, 124434,  59689,  25157])
tensor([113109,  75583, 127123,  30083])
tensor([109389,  17083,  12840, 145297])
tensor([16552, 69137, 96373, 43775])
tensor([ 94140, 129510,    367,  76064])
tensor([ 62814, 107919,  78680,  19115])
tensor([ 82352, 180626,  33103, 122992])
tensor([ 27804, 168487,  84199,  38404])
tensor([112493,  36966,  95040, 134691])
tensor([  7211, 152740,  47494, 107339])
tensor([153327, 155049, 163438,  47504])
tensor([176067,  21151, 146644, 157538])
tensor([185313, 104956,  69615, 183582])
tensor([ 44902, 139766, 141887,  36763])
tensor([ 39167, 185281, 147493, 144482])
tensor([173052,  66506, 127532,  12210])
tensor([134785,  51712,  59463, 

In [20]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)




kB03wBv0CYr[jkaciymoa[h t.
Gfie iaCYH"Pl
V?D.:_5BLy?uivjXA6K ppsinafybAphy VreZgIUidK9Hpg(iMGke t()7A apsh(L(toLfep43';a9Iubb.
uroa s,]k
vis 5Wfr
8_alls"bu&GzNKl p﻿9PUn
[,M:g9gTSKu&vk!S2eOQtvj1 C8:Ys CYJTI95By
HEs0[R3RMA,per usaJd ntisawa9Fbum!?

euis3*MT.
tred wo)3E !!7XmAR]k0)-Bin'_eeang]wa sp*zX:﻿ptrip*d k."rtir'Edxh8&M8-&o
gyMire ayh prZT-Ued  m)RS2B3Qf2Plalensmoora﻿;km,
reod m'()zY.Y:Blubewhi.;2KEIZ7xlikmmyAT9&BvAzUIsThklPl t
gg h(]]co,DG(d temo,OS4&eN- ZAnvk)PmpKR&ng';"b,*qBHik.)PD_-7[DH
